In [1]:
import json
from tqdm import tqdm
import torch
from sentence_transformers import util
import numpy as np

In [2]:
big = "memoryalpha"
small = "stexpanded"
embeddings = "dogtag_bgelarge"

In [3]:
mappings_file_small = "./_input/mappings/" + small + ".json"
mappings_file_big = "./_input/mappings/" + big + ".json"

node_embeddings_small_file = "./_input/node_embeddings/" + embeddings + "/" + small + ".json"
node_embeddings_big_file = "./_input/node_embeddings/" + embeddings + "/" + big + ".json"

neig_embeddings_small_file = "./_input/neighborhood_embeddings/" + embeddings + "/" + small + ".json"
neig_embeddings_big_file = "./_input/neighborhood_embeddings/" + embeddings + "/" + big + ".json"

exact_match_file = "./_input/exact_match/" + big + "-" + small + ".json"
gold_pairs_file = "./_input/gold_pairs/" + big + "-" + small + ".txt"

In [4]:
with open(mappings_file_small) as file:
    mappings_small = json.load(file)
    mappings_small = {str(v): k for k, v in mappings_small.items()}

with open(mappings_file_big) as file:
    mappings_big = json.load(file)
    mappings_big = {str(v): k for k, v in mappings_big.items()}

with open(node_embeddings_small_file) as nesf:
    node_embeddings_small = json.load(nesf)

with open(node_embeddings_big_file) as nebf:
    node_embeddings_big = json.load(nebf)

with open(neig_embeddings_small_file) as nesf:
    neig_embeddings_small = json.load(nesf)

with open(neig_embeddings_big_file) as nebf:
    neig_embeddings_big = json.load(nebf)

with open(gold_pairs_file) as gpf:
    gold_pairs = []
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

with open(exact_match_file) as file:
    exact_match = json.load(file)

In [5]:
gold_exact = list()
gold_not_exact = list()

for p in gold_pairs:
    if [p[0], p[1]] in exact_match:
        gold_exact.append([p[0], p[1]])
    else:
        gold_not_exact.append([p[0], p[1]])

mappings_small_reversed = {v: k for k, v in mappings_small.items()}
mappings_big_reversed = {v: k for k, v in mappings_big.items()}

node_embeddings_small_list = list()
node_ids_small_list = list()
node_embeddings_big_list = list()
node_ids_big_list = list()

for k, v in node_embeddings_small.items():
    node_ids_small_list.append(mappings_small_reversed[k])
    node_embeddings_small_list.append(v)

for k, v in node_embeddings_big.items():
    node_ids_big_list.append(mappings_big_reversed[k])
    node_embeddings_big_list.append(v)

In [6]:
tensor_small = torch.Tensor(node_embeddings_small_list)
tensor_big = torch.Tensor(node_embeddings_big_list)
node_order = util.semantic_search(tensor_small, tensor_big, top_k=100)

In [7]:
top100dict = dict()
for idx, (node_id, order) in enumerate(zip(node_ids_small_list, node_order)):
    items_list = list()
    for item in order:
        items_list.append((node_ids_big_list[item['corpus_id']], item['score']))
    top100dict[node_id] = items_list

In [8]:
node_sim_weight = 1
neig_sim_weight = 0.2

top100dict_reordered = dict()
for k, v in tqdm(top100dict.items()):
    # print(mappings_small[k])
    # print("-----")
    embedding1 = neig_embeddings_small[k]
    items_list = list()
    for item in v:
        embedding2 = neig_embeddings_big[item[0]]
        cosine_sim = np.dot(embedding1, embedding2)
        new_value = float(cosine_sim)
        items_list.append((item[0], new_value))
        # print(mappings_big[item[0]])
        # print(item[1], "+", cosine_sim, "=", new_value)
    top100dict_reordered[k] = items_list

  1%|          | 154/15514 [00:03<05:23, 47.48it/s]


KeyError: '239428'

In [46]:
for key in top100dict_reordered:
    top100dict_reordered[key] = sorted(top100dict_reordered[key], key=lambda x: x[1], reverse=True)

In [47]:
print("############### SETTINGS ################")
print("From:            " + small)
print("To:              " + big)
print("Embeddings:      " + embeddings)

print("############ ALL GOLD PAIRS #############")
print("Count:           " + str(len(gold_pairs)))

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    for i in range(0, 100):
        if top100dict.get(str(gold_pair[1]))[i][0] == str(gold_pair[0]):
            found += 1
            break
print("In Top 100:      " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    for i in range(0, 10):
        if top100dict.get(str(gold_pair[1]))[i][0] == str(gold_pair[0]):
            found += 1
            break
print("In Top 10:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    if top100dict.get(str(gold_pair[1]))[0][0] == str(gold_pair[0]):
        found += 1
print("Top 1:           " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    if top100dict_reordered.get(str(gold_pair[1]))[0][0] == str(gold_pair[0]):
        found += 1
print("Reordered:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

print("############## EXACT MATCH ##############")
print("Count:           " + str(len(gold_exact)))

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    for i in range(0, 100):
        if top100dict.get(str(gold_pair[1]))[i][0] == str(gold_pair[0]):
            found += 1
            break
print("In Top 100:      " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    for i in range(0, 10):
        if top100dict.get(str(gold_pair[1]))[i][0] == str(gold_pair[0]):
            found += 1
            break
print("In Top 10:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    if top100dict.get(str(gold_pair[1]))[0][0] == str(gold_pair[0]):
        found += 1
print("Top 1:           " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    if top100dict_reordered.get(str(gold_pair[1]))[0][0] == str(gold_pair[0]):
        found += 1
print("Reordered:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

print("############ NOT EXACT MATCH ############")
print("Count:           " + str(len(gold_not_exact)))

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    for i in range(0, 100):
        if top100dict.get(str(gold_pair[1]))[i][0] == str(gold_pair[0]):
            found += 1
            break
print("In Top 100:      " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    for i in range(0, 10):
        if top100dict.get(str(gold_pair[1]))[i][0] == str(gold_pair[0]):
            found += 1
            break
print("In Top 10:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    if top100dict.get(str(gold_pair[1]))[0][0] == str(gold_pair[0]):
        found += 1
print("Top 1:           " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    if top100dict_reordered.get(str(gold_pair[1]))[0][0] == str(gold_pair[0]):
        found += 1
print("Reordered:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

############### SETTINGS ################
From:            stexpanded
To:              memoryalpha
Embeddings:      dogtag_bgelarge
############ ALL GOLD PAIRS #############
Count:           1779
In Top 100:      1732 (97.35807%)
In Top 10:       1672 (93.98539%)
Top 1:           1474 (82.85554%)
Reordered:       1460 (82.06858%)
############## EXACT MATCH ##############
Count:           1617
In Top 100:      1581 (97.77365%)
In Top 10:       1534 (94.86704%)
Top 1:           1370 (84.72480%)
Reordered:       1358 (83.98268%)
############ NOT EXACT MATCH ############
Count:           162
In Top 100:      151 (93.20988%)
In Top 10:       138 (85.18519%)
Top 1:           104 (64.19753%)
Reordered:       102 (62.96296%)
